In [1]:
import keras
from keras.preprocessing import image
from keras import layers, models, optimizers
from keras.applications import VGG16, ResNet50, DenseNet121,AlexNet



Using TensorFlow backend.


In [2]:
base_model = VGG16(input_shape=(224, 224, 3), weights='imagenet', include_top=True)
base_model.summary()



553467904/553467096 [==============================] - 167s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
__________________________________________________________

In [3]:
X = base_model.layers[-3].output
X = layers.Dense(512, activation='softmax')(X)
X = layers.Dense(512, activation='softmax')(X)
X = layers.Dense(102, activation='softmax')(X)
base_model = models.Model(inputs=base_model.inputs, outputs=X)

base_model.summary()



Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [4]:
for i in range(len(base_model.layers)-3):
	base_model.layers[i].trainable = False
base_model.summary()


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [7]:
import os
DATA_FOLDER = "E:/CalTech101"
# DATA_FOLDER = "CalTech101"
TRAIN_PATH = os.path.join(DATA_FOLDER, "training") # Path for training data
VALID_PATH = os.path.join(DATA_FOLDER, "validation") # Path for validation data
NUMBER_OF_CLASSES = len(os.listdir(TRAIN_PATH)) # Number of classes of the dataset



In [10]:
batch_size=8
EPOCHS=2
datagen = image.ImageDataGenerator(preprocessing_function=keras.applications.vgg16.preprocess_input) # creating an instance of the data generator
train_generator = datagen.flow_from_directory(TRAIN_PATH, target_size=(224, 224), batch_size=batch_size) # creating the generator for training data
valid_generator = datagen.flow_from_directory(VALID_PATH, target_size=(224, 224), batch_size=batch_size) # creating the generator for validation data

base_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = base_model.fit_generator(train_generator,validation_data=valid_generator, epochs=EPOCHS, steps_per_epoch=len(train_generator) / batch_size, validation_steps=len(valid_generator))

print(max(history.history['val_accuracy']))
print(history.history)

Found 7315 images belonging to 102 classes.
Found 1829 images belonging to 102 classes.
Epoch 1/2
115/114 [==============================] - 1205s 10s/step - loss: 4.5975 - accuracy: 0.0787 - val_loss: 4.5605 - val_accuracy: 0.0962
Epoch 2/2
115/114 [==============================] - 708s 6s/step - loss: 4.5409 - accuracy: 0.1098 - val_loss: 4.4040 - val_accuracy: 0.0962
0.09622744470834732
{'val_loss': [4.560450077056885, 4.404014587402344], 'val_accuracy': [0.09622744470834732, 0.09622744470834732], 'loss': [4.597537917778141, 4.540933919989544], 'accuracy': [0.078688525, 0.10978261]}
